<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [7]:
%run "../../xUtilz/envz_only_loader.ipynb"
%run "../../xUtilz/utilz_only_loader.ipynb"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [8]:
db = pd.read_csv( "../../datasets/LG/IN/181112_Referral_Reasons.csv" )

In [9]:
db["CHV Name"] = "CHV " +  pd.Series( pd.factorize( db["CHV Name"])[0] + 1).astype( str)
db["Patient Name"] = "Patient " +  pd.Series( pd.factorize( db["Patient Name"])[0] + 1).astype( str)
db.drop( "CHV Phone", axis=1, inplace=True)

In [10]:
reasons_groupz = { 
    "Neonates" : [ "months", "umbilical", "umbllical"], 
    "Pregnancy" : ["delivery"], 
    "U5 Danger Signs" : [ "Danger", "Diarrhoea", "Diarrhea", "Dairrhea", "Coughing", "Cough", "Fever","Pneumonia", "Pnuemonia",
                        "Pheumonia", ],
    "HIVST" : ["reactive", "partner", "intimate"], 
    "Unsupported Cases" : []
}

db["Category Referral Reason"] = db["Referral Reason"].apply( lambda x: searchString(x, reasons_groupz) ) 

var_bucket_reasons =  "Category Referral Reason"
var_all_reasons = "All"
var_Unsupported = "Unsupported Cases"
var_HIVST = "HIVST"
var_Display_Colz = ["Date of referral", "CHV Name", "Patient Name" ,"Referral Reason", "Reason for Referral"]

In [5]:
# 1. set Date of referral to a datetime type 
db["Date of referral"] = pd.to_datetime( db["Date of referral"], format="%d-%b-%y") # inplace=True)

# 2. Extract Month and year categories 
db["Year"] = db["Date of referral"].dt.year 
db["Month"] = db["Date of referral"].dt.strftime('%b-%y') 
db["Day_of_Week"] = db["Date of referral"].dt.weekday_name.str[:3] 

db.sort_values( by='Date of referral', inplace=True)

LAST_UPDATED = db["Date of referral"].max().strftime( '%d-%b-%Y')

In [13]:
db.columns

Index(['Date of referral', 'CHV Name', 'Patient Name', 'Referral Reason',
       'Category Referral Reason'],
      dtype='object')

In [16]:
display(HTML('<b>Number of Records Table - Digital Referrals </b>'))
t = pd.pivot_table( db, index=var_bucket_reasons, values=["Patient Name"], aggfunc='count', margins=True ).T
t["Total Per. CHV"] = len( db ) / len( db["CHV Name"].unique() )
#t[ "Total per CHV per Month"] = (len( db ) / len( db["CHV Name"].unique() ) )/ (db["Date of referral"].max().to_period('M') - db["Date of referral"].min().to_period('M'))
t['Number of CHVs'] =  len( db["CHV Name"].unique() ) 
t['Records per Client'] =  len(db)/len( db["Patient Name"].unique() )
t.round(2)


Category Referral Reason,HIVST,Neonates,No Response,Pregnancy,U5 Danger Signs,Unsupported Cases,All,Total Per. CHV,Number of CHVs,Records per Client
Patient Name,19.0,7.0,11.0,1.0,42.0,53.0,133.0,5.32,25,1.14


In [17]:
t.columns

Index(['HIVST', 'Neonates', 'No Response', 'Pregnancy', 'U5 Danger Signs',
       'Unsupported Cases', 'All', 'Total Per. CHV', 'Number of CHVs',
       'Records per Client'],
      dtype='object', name='Category Referral Reason')

In [25]:
for i in t.columns[6:]:
    print( "%s : %s   ]]]]"% (i, t[i][0]))

All : 133.0   ]]]]
Total Per. CHV : 5.32   ]]]]
Number of CHVs : 25   ]]]]
Records per Client : 1.13675213675   ]]]]
